In [ ]:
import os
import pandas as pd
import shutil
import uuid

 
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.python.compiler.tensorrt import trt_convert as trt
from tensorflow.python.saved_model import signature_constants, tag_constants
from tensorflow.python.framework import convert_to_constants
from tensorflow.keras.models import load_model

import cv2
import numpy as np
import time
import math
from tensorflow.keras.models import load_model

In [ ]:
trt_model_dir = f"./trt_models"

In [ ]:
# load saved model
saved_model_loaded = tf.saved_model.load(trt_model_dir, tags=[tag_constants.SERVING])
signature_keys = list(saved_model_loaded.signatures.keys())
print(signature_keys)

infer = saved_model_loaded.signatures['serving_default']
print(infer.structured_outputs)

In [ ]:
video=cv2.VideoCapture(1)

modelFile = "../models/res10_300x300_ssd_iter_140000.caffemodel"
configFile = "../models/deploy.prototxt.txt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

labels_dict = {0:'Anger', 1:'Fear', 2:'Happy', 3:'Neutral', 4:'Sad', 5:'Surprise'}

counter = 0
while True:
    counter += 1
    ret,frame=video.read()
    frame = cv2.flip(frame, 1)
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)),
                                 1.0, (300, 300), (104.0, 117.0, 123.0))
    net.setInput(blob)
    faces = net.forward()
    h, w = frame.shape[:2]
    for i in range(faces.shape[2]):
        confidence = faces[0, 0, i, 2]
        if confidence > 0.5:
            box = faces[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x1, y1) = box.astype("int")
            sub_face_img = frame[y:y1, x:x1]
            try:
                resized = cv2.resize(sub_face_img, (224, 224))
                reshaped = np.reshape(resized, (1, 224, 224, 3))
             
                
                tensor = tf.convert_to_tensor(reshaped)
                labeling = infer(tensor)
                label = labeling['dense_1'].numpy().argmax()
                

                cv2.rectangle(frame, (x, y), (x1, y1), (0, 0, 255), 1)
                cv2.rectangle(frame, (x, y), (x1, y1), (50, 50, 255), 2)
                # cv2.rectangle(frame, (x, y - 100), (x1, y), (50, 50, 255), -1)
                cv2.rectangle(frame, (x, y - 40), (x1, y), (50, 50, 255), -1)

                cv2.putText(frame, labels_dict[label], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

            except Exception as e:
                print(str(e))
    frame = cv2.resize(frame, (1920, 1080), fx=0, fy=0, interpolation=cv2.INTER_CUBIC)
    cv2.imshow("Frame",frame)
    k=cv2.waitKey(1)
    if k==ord('q'):
        break

video.release()
cv2.destroyAllWindows()